In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow as tf
print(tf.test.is_built_with_cuda())

with tf.Session() as sess:
    devices = sess.list_devices()
    print(devices)

#watch -n 2 nvidia-smi
#watch -d -n 0.5 nvidia-smi

# Basic Operations with variable as graph input
# The value returned by the constructor represents the output
# of the Variable op. (define as input when running session)
# tf Graph input
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

# Define some operations
add = tf.add(a, b)
mul = tf.multiply(a, b)

# Launch the default graph.
with tf.Session() as sess:
    # Run every operation with variable input
    print ("Addition with variables: %i" % sess.run(add, feed_dict={a: 2, b: 3}))
    print ("Multiplication with variables: %i" % sess.run(mul, feed_dict={a: 2, b: 3}))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Addition with variables: 5
Multiplication with variables: 6


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import combinations as cs
import time

num_situ = 20
batch_size = 100
two_list = [[[2**x] for x in range(0,num_situ)] for j in range(batch_size)]
print(time.ctime())
#two_list =[[ [2], [3], [5], [7], [11], [13], [17], [19], [23], [29], [31], [37], [41], [43], [47], [53], [59], [61], [67], [71]] for j in range(batch_size)]
#glist = ['aay','Ama','Ance','Antp','apt','Blimp-1','bmm','bowl','brk','Btk29A','bun','cad','CenG1A','CG10479','CG11208','CG14427','CG17724','CG17786','CG43394','CG8147','cnc','croc','Cyp310a1','D','dan','danr','Dfd','disco','Doc2','Doc3','dpn','edl','ems','erm','Esp','E(spl)m5-HLH','eve','exex','fj','fkh','ftz','gk','gt','h','hb','hkb','htl','Ilp4','ImpE2','ImpL2','ken','kni','knrl','Kr','lok','Mdr49','Mes2','MESR3','mfas','Nek2','NetA','noc','nub','numb','oc','odd','peb','prd','pxb','rau','rho','run','sna','srp','tkv','tll','toc','Traf4','trn','tsh','twi','zen','zen2','zfh1']
glist = ['danr', 'CG14427', 'dan', 'CG43394', 'ImpL2', 'Nek2', 'CG8147', 'Ama', 'Btk29A', 'trn', 'numb', 'prd', 'brk', 'tsh', 'pxb', 'dpn', 'ftz', 'Kr', 'h', 'eve', 'Traf4', 'run', 'Blimp-1', 'lok', 'kni', 'tkv', 'MESR3', 'odd', 'noc', 'nub', 'Ilp4', 'aay', 'twi', 'bmm', 'hb', 'toc', 'rho', 'CG10479', 'gt', 'gk', 'apt', 'D', 'sna', 'NetA', 'Mdr49', 'fj', 'Mes2', 'CG11208', 'Doc2', 'bun', 'tll', 'Cyp310a1', 'Doc3', 'htl', 'Esp', 'bowl', 'oc', 'ImpE2', 'CG17724', 'fkh', 'edl', 'ems', 'zen2', 'CG17786', 'zen', 'disco', 'Dfd', 'mfas', 'knrl', 'Ance', 'croc', 'rau', 'cnc', 'exex', 'cad', 'Antp', 'erm', 'ken', 'peb', 'srp', 'E(spl)m5-HLH', 'CenG1A', 'zfh1', 'hkb']
bdtnp_bin = pd.read_csv('binarized_bdtnp.csv')
bdtnp = bdtnp_bin.values.astype(np.int32)
#matrix1 = tf.constant(bdtnp)
#matrix2 = tf.constant(two_list)

# from generator
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])
def generator():
    for tup in cs(glist, num_situ):
        yield bdtnp_bin[np.reshape(tup, -1)].values.astype(np.int32)

with tf.device('/GPU:0'):
    dataset = tf.data.Dataset().from_generator(generator,
                                           output_types= tf.int32, 
                                           output_shapes=(tf.TensorShape([3039, num_situ]))).batch(batch_size).prefetch(60000)
    iter = dataset.make_initializable_iterator()
    X = iter.get_next()
    Y = tf.placeholder(tf.int32, shape=(batch_size, num_situ,1))
   
    #len(np.unique(np.matmul(bdtnp, two_list)))
    # Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs and return matrix multiplication.
    # Using Y we get a unique number for each of the 3039 rows.
    product = tf.matmul(X, Y) # product.shape == (10, 3039, 1)
    #Move to vector
    un_2 = tf.squeeze(product) # un_2.shape == (10, 3039)
    
    """
    #Check how many unique elements
    @tf.contrib.eager.defun
    def func(tensors):
        y, idx = tf.unique(tensors) #, dtype=tf.int32)
        return tf.size(y)

    #un_3 = tf.unique(un_2)
    un_3 = tf.map_fn(func, un_2, infer_shape=False)
    """

    #Use the following line to count appearance of the same unique number in different rows
    #flat = tf.reshape(un_2, [-1])
    #y_vals, idx = tf.unique(flat) # Since bdtnp is highly sparsed we need to keep only a small set of indices
    one_hot_t = tf.one_hot(un_2, 512) # one_hot_t.shape == (10, 3039, 639) #639, y_vals
    count_all = tf.reduce_sum(one_hot_t, axis=1) # count_all.shape == (10, 3039)
    count_ones = tf.ones_like(count_all)
    count_zeros = tf.zeros_like(count_all)
    exact_count = tf.where(tf.equal(count_all, 1.), count_ones, count_zeros)
    sum_count = tf.reduce_sum(exact_count, axis=1) # sum_count.shape == (10,)

    """
    ta = tf.TensorArray(dtype=tf.int32, size=100)
    def body(i):
        ta.write(i, tf.unique(un_2[i,:]))
    
    i = tf.constant(0)
    cond = lambda i: tf.less(i, 100)
    ta_final = tf.while_loop(cond, body, [i])
    # get the final result
    ta_final_result = ta.stack()
    """
# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of threes ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
with tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
    sess.run(iter.initializer)
    best_result = 0
    # `sess.graph` provides access to the graph used in a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>.
    #writer = tf.summary.FileWriter("./log/", sess.graph)
    for i in range(10):
        #if(i % 10 == 0):
        print(f'Best result: {best_result}')
        result = max(sess.run(sum_count, feed_dict={Y: two_list}))
        if(result > best_result):
             best_result = result
    #writer.close()

print(time.ctime(), f'Best result: {best_result}')

Thu Dec  6 23:13:03 2018
Best result: 0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Best result: 10.0
Thu Dec  6 23:13:18 2018 Best result: 10.0
